In [1]:
import sys
from pathlib import Path
from os.path  import join
from src.config import DATA_DIR, CONFIG_DIR
from omegaconf import OmegaConf
from src.external.hptr.src.data_modules.agent_centric import AgentCentricPreProcessing
from src.external.hptr.src.data_modules.ac_global import AgentCentricGlobal
from src.mimolm import InputProjections, EarlyFusionEncoder, MotionDecoder
import torch
import torch.nn.functional as F
import lightning as pl

# Add the project root to sys.path
project_root = Path().resolve().parent  # Adjust as needed to point to the root folder
sys.path.append(str(project_root))

print(Path.cwd())  # Check if the path is added

2025-02-04 22:17:07.128 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: /home/harshavardhan-patil/Work/Projects/mimolm
/home/harshavardhan-patil/Work/Projects/mimolm/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/harshavardhan-patil/Work/Projects/mimolm/notebooks


In [2]:
from src.external.hptr.src.data_modules.data_h5_av2 import DataH5av2
from src.mimolm import MimoLM

data_module = DataH5av2(DATA_DIR / "interim" / "train")
data_module.setup(stage="fit")
train_loader = data_module.train_dataloader()
val_loader = data_module.val_dataloader()

model = MimoLM(data_size=data_module.tensor_size_val)

trainer = pl.Trainer(fast_dev_run=1,)
trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/harshavardhan-patil/Work/Projects/mimolm/venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will t

Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  1.51it/s]

/home/harshavardhan-patil/Work/Projects/mimolm/venv/lib/python3.12/site-packages/lightning/pytorch/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 2. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Epoch 0: 100%|██████████| 1/1 [00:01<00:00,  0.69it/s, val_loss=4.630, train_loss=5.140]

`Trainer.fit` stopped: `max_steps=1` reached.


Epoch 0: 100%|██████████| 1/1 [00:01<00:00,  0.69it/s, val_loss=4.630, train_loss=5.140]


In [26]:
import torch.nn.functional as F
F.softmax(torch.rand((16, 55, 169))[:, -1], dim=-1).argmax(dim=1)

tensor([129, 111, 137, 136,  95, 108,  83,   1,  88, 156, 126,  43,  80,  70,
         73, 153])

In [4]:
from src.modeling.modules import lm_utils

_, pos_bins, verlet_wrapper = lm_utils.create_vocabulary(max_delta=9, n_quantization_bins=128, n_verlet_steps=13)

In [5]:
pos_bins

tensor([[-9.0000, -8.8583, -8.7165, -8.5748, -8.4331, -8.2913, -8.1496, -8.0079,
         -7.8661, -7.7244, -7.5827, -7.4409, -7.2992, -7.1575, -7.0157, -6.8740,
         -6.7323, -6.5906, -6.4488, -6.3071, -6.1654, -6.0236, -5.8819, -5.7402,
         -5.5984, -5.4567, -5.3150, -5.1732, -5.0315, -4.8898, -4.7480, -4.6063,
         -4.4646, -4.3228, -4.1811, -4.0394, -3.8976, -3.7559, -3.6142, -3.4724,
         -3.3307, -3.1890, -3.0472, -2.9055, -2.7638, -2.6220, -2.4803, -2.3386,
         -2.1969, -2.0551, -1.9134, -1.7717, -1.6299, -1.4882, -1.3465, -1.2047,
         -1.0630, -0.9213, -0.7795, -0.6378, -0.4961, -0.3543, -0.2126, -0.0709,
          0.0709,  0.2126,  0.3543,  0.4961,  0.6378,  0.7795,  0.9213,  1.0630,
          1.2047,  1.3465,  1.4882,  1.6299,  1.7717,  1.9134,  2.0551,  2.1969,
          2.3386,  2.4803,  2.6220,  2.7638,  2.9055,  3.0472,  3.1890,  3.3307,
          3.4724,  3.6142,  3.7559,  3.8976,  4.0394,  4.1811,  4.3228,  4.4646,
          4.6063,  4.7480,  

In [8]:
pos_bins[0][[26, 32, 84]]

tensor([-5.3150, -4.4646,  2.9055])

In [7]:
verlet_wrapper

tensor([-6., -5., -4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.])

In [29]:
#lasttoken = 2, 8, 2
verlet_wrapper[torch.randint(low=0, high=13, size=[16, 3])[:, 1:].unflatten(dim=0, sizes=(2, 8))]


tensor([[[-3.,  4.],
         [-2.,  5.],
         [-1., -4.],
         [ 2., -3.],
         [-5., -6.],
         [ 0., -1.],
         [ 4.,  6.],
         [ 6., -2.]],

        [[ 6.,  0.],
         [ 6., -5.],
         [-3.,  0.],
         [-4.,  2.],
         [ 2., -1.],
         [ 1., -4.],
         [-4., -1.],
         [ 2., -3.]]])

In [20]:
hjask

NameError: name 'hjask' is not defined

In [2]:
from src.external.hptr.src.data_modules.data_h5_av2 import DataH5av2

data_module = DataH5av2(DATA_DIR / "interim" / "train" )
data_module.setup(stage="fit")
dl = data_module.train_dataloader()

In [3]:
from src.mimolm import MimoLM

model = MimoLM(data_size=data_module.tensor_size_train)

Input projections parameters: 0.47M
Encoder parameters: 2.42M
Decoder parameters: 1.63M


In [16]:
from src.modeling.modules.lm_utils import tokenize_motion

#dic = OmegaConf.load(Path(CONFIG_DIR / "model/ac_wayformer.yaml"))
episode = None
fmask = None
prepreprocess = AgentCentricPreProcessing(time_step_current=49, 
                                          data_size=data_module.tensor_size_train, 
                                          n_target=8,
                                          n_other=48,
                                          n_map=512,
                                          mask_invalid=False)

preprocess = AgentCentricGlobal(time_step_current=49,
                                data_size=data_module.tensor_size_train,
                                dropout_p_history=0.15, 
                                add_ohe=True,
                                pl_aggr=False,
                                pose_pe= {"agent": "xy_dir",
                                        "map": "mpa_pl"})

inputproj = InputProjections(hidden_dim=256,
                             agent_attr_dim=preprocess.model_kwargs["agent_attr_dim"],
                             map_attr_dim=preprocess.model_kwargs["map_attr_dim"],
                             n_step_hist=preprocess.model_kwargs["n_step_hist"],
                             n_pl_node=preprocess.model_kwargs["n_pl_node"],
                             add_learned_pe=True,
                             n_layer_mlp=3,
                             mlp_cfg={"end_layer_activation" : True,
                                        "use_layernorm" :  False,
                                        "use_batchnorm" : False,
                                        "dropout_p" : None,})

encoder = EarlyFusionEncoder(hidden_dim=256,
                             tf_cfg={"n_head": 4,
                                     "dropout_p": 0.1,
                                     "norm_first": True,
                                     "bias": True},
                             latent_query={"use_agent_type": False,
                                           "mode_emb": "none", # linear, mlp, add, none
                                           "mode_init": "xavier", # uniform, xavier
                                           "scale": 5.0},
                             n_latent_query=192,
                             n_encoder_layers=2)  

decoder = MotionDecoder()
logsoftmax = torch.nn.LogSoftmax(dim=-1)
loss_fn = torch.nn.NLLLoss()

In [19]:
f = None
femb = None
a = None
l = None
for batch in dl:
    batch = prepreprocess(batch=batch)
    batch = preprocess(batch=batch)
    actuals = tokenize_motion(batch["gt/pos"],
                            decoder.pos_bins, 
                            decoder.verlet_wrapper, 
                            decoder.n_verlet_steps, 
                            decoder.n_time_steps)
    a = actuals
    motion_tokens = torch.cat((batch["ac/target_pos"], batch["gt/pos"]), dim = -2)
    target_types = batch["ac/target_type"]
    input_dict = {
    k.split("input/")[-1]: v for k, v in batch.items() if "input/" in k
    }
    valid = input_dict["target_valid"].any(-1)
    target_emb, target_valid, other_emb, other_valid, map_emb, map_valid = inputproj(target_valid = input_dict["target_valid"], 
              target_attr = input_dict["target_attr"],
              other_valid = input_dict["other_valid"],
              other_attr = input_dict["other_attr"],
              map_valid = input_dict["map_valid"],
              map_attr = input_dict["map_attr"],)
    fused_emb, fused_emb_invalid = encoder(
                target_emb, target_valid, other_emb, other_valid, map_emb, map_valid, input_dict["target_type"], valid
            )
    pred = decoder(motion_tokens, target_types, fused_emb, fused_emb_invalid)
    episode = pred
    loss = loss_fn(
        logsoftmax(pred[:, 50:, :].flatten(0, 1)), 
        actuals.flatten(0, 1).flatten(0, 1).repeat(64))
    print(loss)
    break

torch.Size([2, 8, 110])
tensor(5.1317, grad_fn=<NllLossBackward0>)


In [5]:
episode.shape

torch.Size([1024, 110, 169])

In [6]:
episode[:, 50:, :].flatten(0, 1).shape

torch.Size([61440, 169])

In [ ]:
loss_fn(F.log_softmax(episode[:, 50:, :].flatten(0, 1), dim=-1), a.flatten(0, 1).flatten(0, 1))

In [9]:
a.shape

torch.Size([2, 8, 60])

In [14]:
a.flatten(0, 1).flatten(0, 1).shape

torch.Size([960])

In [7]:
femb.shape

torch.Size([16, 192, 256])

In [8]:
temp = torch.arange(0, 110)
temp.unsqueeze(0).repeat(motion_tokens.shape[1], 1).unsqueeze(0).repeat(motion_tokens.shape[0], 1, 1)

tensor([[[  0,   1,   2,  ..., 107, 108, 109],
         [  0,   1,   2,  ..., 107, 108, 109],
         [  0,   1,   2,  ..., 107, 108, 109],
         ...,
         [  0,   1,   2,  ..., 107, 108, 109],
         [  0,   1,   2,  ..., 107, 108, 109],
         [  0,   1,   2,  ..., 107, 108, 109]],

        [[  0,   1,   2,  ..., 107, 108, 109],
         [  0,   1,   2,  ..., 107, 108, 109],
         [  0,   1,   2,  ..., 107, 108, 109],
         ...,
         [  0,   1,   2,  ..., 107, 108, 109],
         [  0,   1,   2,  ..., 107, 108, 109],
         [  0,   1,   2,  ..., 107, 108, 109]]])

In [9]:
episode.shape

torch.Size([1024, 110, 169])

In [ ]:
episode[1] == episode.reshape((-1, 256))[110:220]

In [11]:
fmask.shape

torch.Size([16, 192])

In [12]:
fused_emb.shape

torch.Size([16, 192, 256])

In [81]:
fmask.shape

torch.Size([6, 192])

In [7]:
from src.modeling.modules.lm_utils import create_vocabulary

create_vocabulary(8, 128, 13)

(tensor([[  0,   0,   0],
         [  1,   0,   1],
         [  2,   0,   2],
         [  3,   0,   3],
         [  4,   0,   4],
         [  5,   0,   5],
         [  6,   0,   6],
         [  7,   0,   7],
         [  8,   0,   8],
         [  9,   0,   9],
         [ 10,   0,  10],
         [ 11,   0,  11],
         [ 12,   0,  12],
         [ 13,   1,   0],
         [ 14,   1,   1],
         [ 15,   1,   2],
         [ 16,   1,   3],
         [ 17,   1,   4],
         [ 18,   1,   5],
         [ 19,   1,   6],
         [ 20,   1,   7],
         [ 21,   1,   8],
         [ 22,   1,   9],
         [ 23,   1,  10],
         [ 24,   1,  11],
         [ 25,   1,  12],
         [ 26,   2,   0],
         [ 27,   2,   1],
         [ 28,   2,   2],
         [ 29,   2,   3],
         [ 30,   2,   4],
         [ 31,   2,   5],
         [ 32,   2,   6],
         [ 33,   2,   7],
         [ 34,   2,   8],
         [ 35,   2,   9],
         [ 36,   2,  10],
         [ 37,   2,  11],
         [ 3